In [1]:
#!pip install opencv-python

In [2]:
import numpy  as np
import matplotlib.pyplot as plt
import glob
from IPython.display import HTML
%matplotlib inline
import cv2

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Activation, Reshape
from keras.layers import LeakyReLU

/home/ubuntu/anaconda/envs/py36-venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Construct Yolo model

In [3]:
keras.backend.set_image_dim_ordering('th')

In [4]:
model = Sequential()

# Block 1
model.add(Conv2D(filters=16, 
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1),
                 input_shape=(3,448,448))
         )
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(filters=32,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(filters=64,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(filters=128,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Block 5
model.add(Conv2D(filters=256,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Block 6
model.add(Conv2D(filters=512,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Block 7
model.add(Conv2D(filters=1024,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))

# Block 8
model.add(Conv2D(filters=1024,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))

# Block 9
model.add(Conv2D(filters=1024,
                 kernel_size=(3, 3),
                 padding="same", 
                 strides=(1, 1))
         )
model.add(LeakyReLU(alpha=0.1))

In [5]:
model.add(Flatten())

model.add(Dense(256))
model.add(Dense(4096))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(1470))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 448, 448)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 224, 224)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
__________

## Load weight for Yolo

In [7]:
from utils import load_weights, yolo_net_out_to_car_boxes, draw_box

In [8]:
load_weights(model,'./models/yolo-tiny.weights')

In [9]:
imagePath = './data/cars/test5.jpg'
image = plt.imread(imagePath)
image_crop = image[30:650,50:,:]
resized = cv2.resize(image_crop,(448,448))

In [10]:
batch = np.transpose(resized,(2,0,1))
batch = 2*(batch/255.) - 1
batch = np.expand_dims(batch, axis=0)
out = model.predict(batch)
out

array([[0.02278489, 0.01114899, 0.04798217, ..., 0.2425352 , 0.15741125,
        0.27026805]], dtype=float32)

In [11]:
labels = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", 
          "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", 
          "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

class_idx = 6 # the index of car in the lable list of tiny yolo

In [12]:
boxes = yolo_net_out_to_car_boxes(out[0], class_idx, threshold = 0.17)
print(boxes[0])

IndexError: list index out of range

In [ ]:
plt.figure(figsize=(80,40))
plt.imshow(draw_box(boxes, plt.imread(imagePath), [[0,1280],[30,650]]))